In [13]:
import sys
!{sys.executable} -m pip install psycopg2-binary
!{sys.executable} -m pip install pandas

In [14]:
import pandas as pd
import psycopg2

In [15]:
PGHOST = "happinesscensus.cnbuz9nrfajq.us-east-1.rds.amazonaws.com"
PGDATABASE = "Happiness"
PGUSER = 'postgres'
PGPASSWORD = '*********'

In [16]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGHOST +" port="+ "5432" +" dbname="+ PGDATABASE +" user=" + PGUSER \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [17]:
conn, cursor = connect()

Connected!


In [18]:
# census
query_for_fetching_from_census_table = """SELECT * FROM census;"""

# happiness
query_for_fetching_from_happiness_table = """SELECT * FROM happiness;"""

# clean_combined_df
query_for_fetching_from_clean_combined_df_table = """SELECT * FROM clean_combined_df;"""

In [19]:
def get_dataframe(cursor, query):
    cursor.execute(query)
    rows = cursor.fetchall()
    
    columns = [ col[0] for col in cursor.description ]
     
    df = pd.DataFrame(rows, columns=columns)
    return df

In [20]:
%%time
df_census = get_dataframe(cursor, query_for_fetching_from_census_table)
df_happiness = get_dataframe(cursor, query_for_fetching_from_happiness_table)
df_clean_combined = get_dataframe(cursor, query_for_fetching_from_clean_combined_df_table)

CPU times: user 735 ms, sys: 329 ms, total: 1.06 s
Wall time: 4.78 s


In [21]:
df_census.head()

,census_id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,income,country
0,00001,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,<=50K,United States
1,00002,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,<=50K,United States
2,00003,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,>50K,United States
3,00004,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,>50K,United States
4,00005,18,Unknown,103497,Some-college,10,Never-married,Unknown,Own-child,White,Female,0,0,30,<=50K,United States


In [22]:
df_census.shape

(48842, 16)

In [23]:
df_happiness.head()

,report_id,census_id,country,ladder_score,gdp,social_support,life_expectancy,freedom_of_choice,generosity,perceived_corruption,ladder_score_dystopia,explained_by_gdp,explained_by_social_support,explained_by_life_expectancy,explained_by_freedom_of_choice,explained_by_generosity,explained_by_perceived_corruption,dystopiaplusresidual
0,00001,00001,Afghanistan,2.523,7.695,0.463,52.493,0.382,-0.102,0.924,2.430,0.370,0.000,0.126,0.000,0.122,0.010,1.895
1,00002,00002,Albania,5.117,9.520,0.697,68.999,0.785,-0.030,0.901,2.430,1.008,0.529,0.646,0.491,0.168,0.024,2.250
2,00003,00003,Algeria,4.887,9.342,0.802,66.005,0.480,-0.067,0.752,2.430,0.946,0.765,0.552,0.119,0.144,0.120,2.242
3,00004,00004,Argentina,5.929,9.962,0.898,69.000,0.828,-0.182,0.834,2.430,1.162,0.980,0.646,0.544,0.069,0.067,2.461
4,00005,00005,Armenia,5.283,9.487,0.799,67.055,0.825,-0.168,0.629,2.430,0.996,0.758,0.585,0.540,0.079,0.198,2.127


In [24]:
df_happiness.shape

(144, 18)

In [25]:
df_clean_combined.head()

,id,age,workclass,education,marital_status,occupation,race,sex,hours_per_week,income,country,ladder_score,gdp,social_support,life_expectancy,freedom_of_choice,generosity,perceived_corruption
0,00001,25,Private,11th,Never-married,Machine-op-inspct,Black,Male,40,<=50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698
1,00002,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,White,Male,50,<=50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698
2,00003,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,White,Male,40,>50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698
3,00004,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Black,Male,40,>50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698
4,00005,18,Unknown,Some-college,Never-married,Unknown,White,Female,30,<=50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698


In [26]:
# show column names
df_happiness.columns.tolist() 

['report_id',
 'census_id',
 'country',
 'ladder_score',
 'gdp',
 'social_support',
 'life_expectancy',
 'freedom_of_choice',
 'generosity',
 'perceived_corruption',
 'ladder_score_dystopia',
 'explained_by_gdp',
 'explained_by_social_support',
 'explained_by_life_expectancy',
 'explained_by_freedom_of_choice',
 'explained_by_generosity',
 'explained_by_perceived_corruption',
 'dystopiaplusresidual']

In [27]:
# show column names
df_census.columns.tolist()

['census_id',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'income',
 'country']

In [28]:
# show column names
df_clean_combined.columns.tolist()

['id',
 'age',
 'workclass',
 'education',
 'marital_status',
 'occupation',
 'race',
 'sex',
 'hours_per_week',
 'income',
 'country',
 'ladder_score',
 'gdp',
 'social_support',
 'life_expectancy',
 'freedom_of_choice',
 'generosity',
 'perceived_corruption']

In [29]:
# get count of null values for each column
[[column,df_clean_combined[column].isnull().sum()] for column in df_clean_combined.columns]

[['id', 0],
 ['age', 0],
 ['workclass', 0],
 ['education', 0],
 ['marital_status', 0],
 ['occupation', 0],
 ['race', 0],
 ['sex', 0],
 ['hours_per_week', 0],
 ['income', 0],
 ['country', 0],
 ['ladder_score', 0],
 ['gdp', 0],
 ['social_support', 0],
 ['life_expectancy', 0],
 ['freedom_of_choice', 0],
 ['generosity', 0],
 ['perceived_corruption', 0]]

In [30]:
# us get_dummies to chane change sex to number
clean_combined_encoded = pd.get_dummies(df_clean_combined, columns=["sex"])
clean_combined_encoded.head()

,id,age,workclass,education,marital_status,occupation,race,hours_per_week,income,country,ladder_score,gdp,social_support,life_expectancy,freedom_of_choice,generosity,perceived_corruption,sex_Female,sex_Male
0,00001,25,Private,11th,Never-married,Machine-op-inspct,Black,40,<=50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698,0,1
1,00002,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,White,50,<=50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698,0,1
2,00003,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,White,40,>50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698,0,1
3,00004,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Black,40,>50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698,0,1
4,00005,18,Unknown,Some-college,Never-married,Unknown,White,30,<=50K,United States,6.951,11.023,0.920,68.200,0.837,0.098,0.698,1,0
